## Working Code

In [1]:
import json
import uuid
from utils import generate_embeddings, get_portkey_client
from pdf_reader import extract_text_from_pdf
from vector_store import initialize_vector_store, add_to_vector_store, create_schema

pdf_path = "ask_hp_faq.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

In [2]:
def chunk_text(text):
    return text.split(". ")

pdf_text_inputs = chunk_text(pdf_text)
vector_store_client = initialize_vector_store()

# get port key client
client = get_portkey_client()

# create FAQ schema
create_schema(vector_store_client, "FAQ")

# get a FAQ collection bucket
collection = vector_store_client.collections.get("FAQ")

for text in pdf_text_inputs:
    gen_uuid = uuid.uuid4()
    embedding_response = client.embeddings.create(input=text, model="text-embedding-3-large", encoding_format="float").data[0].embedding
    collection.data.insert(uuid=gen_uuid.__str__(), properties={"text": text}, vector=embedding_response)    
    


/Users/cecile.yang/Library/Caches/pypoetry/virtualenvs/oncall-agent-CWRHTJmz-py3.13/lib/python3.13/site-packages/portkey_ai/api_resources/base_client.py:154: PydanticDeprecatedSince20: The `construct` method is deprecated; use `model_construct` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  self.allHeaders = self._build_headers(Options.construct())


Collection 'FAQ' has been deleted.
_CollectionConfig(name='FAQ', description=None, generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'), _Property(name='file_path', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorize

In [3]:
query = "I was the discovery-platform oncall and found the `[getCollectionConfig] - error deserializing please check runtime and fix config` log is the top 1 in feed-service. By checking the git commit history, the code was added by Ujjwal a year ago. Can you help check if anything changed recently which caused this log and fix it?"

query_embedding = client.embeddings.create(input=query, model="text-embedding-3-large", encoding_format="float")

In [4]:
new_collection = vector_store_client.collections.get("FAQ")

In [5]:
result = new_collection.query.near_vector(near_vector=query_embedding.data[0].embedding, limit=5, filters=None)

In [9]:
relevant_context = '\n\n'.join([obj.properties['text'] for obj in result.objects])

prompt = f"""
Please answer this question: {query}, based on the following context, and give the context that you used in the answer as part of the output.

{relevant_context}
"""

# print(prompt)

In [7]:
completion = client.chat.completions.create(
    model="gpt-4o-mini", # try a better model
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

In [10]:
# print(completion.choices[0].message.content)